# Extracting data from a website using xpath and scrapy and saving it in JSON format

In [8]:
import scrapy
import logging

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.7.6'

In [10]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [11]:
import json


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('Doctors1.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [12]:

class QuotesSpider(scrapy.Spider):
    name = "quotes1"
    start_urls = [
        'http://www.burjeel.com/abu-dhabi/doctors/'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'Doctors1.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        doctors_name = response.xpath("/html/body/div[3]/div/div[4]/div/div/a[1]/h4/text()").extract()
        Position = response.xpath("/html/body/div[3]/div/div[4]/div/div/a[2]/h5/text()").extract()
        for (d,p) in zip(doctors_name, Position):
            scraped_info = {
                # key:value
                'Doctor': d,
                'Position': p}
            yield scraped_info

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-04-04 23:20:09 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-04 23:20:09 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-04-04 23:20:09 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-04-04 23:20:09 [scrapy.crawler] INFO: Overridden settings:
{'FEED_FORMAT': 'json',
 'FEED_URI': 'Doctors1.json',
 'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [7]:
import pandas as pd
dfjson = pd.read_json('Doctors1.json')
dfjson

,Doctor,Position
0,Dr. Yassin Ibrahim M. El-Shahat,Consultant - Nephrology / Chief Medical Officer
1,Prof. Dr. Iyad Hassan,Consultant - General and Endocrine Surgeon/ He...
2,Dr. Dipak Kamble,Specialist Radiologist
3,Dr.Mostafa Kamal Kotb Ali,Specialist - Orthopedic Surgery
4,Dr. Hussein Nasser Matlik,Consultant - Pediatric Neurology and Head of D...
...,...,...
202,Mr. Chandrashekar Nandi,Physiotherapist & Sports Performance Enhanceme...
203,Dr. Radha Panchapakesan,Specialist - Anesthesiology
204,Joseph Basil C. Ruadap,Physiotherapy
205,Ms. Sikha Anna Sunny,Physiotherapy
